**This program will parse your data of midi files into data readable by the ai**

Import

In [ ]:
!pip install py_midicsv
!pip install mido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 2.9 MB/s eta 0:00:00


In [ ]:
!rm -rf /content/music/

In [ ]:
!cp /content/drive/MyDrive/0.zip /content/music.zip

**Upload your zip file of midi files (all in root of zip) and rename to music.zip also create a folder under the same name in the content folder**

In [ ]:
!unzip /content/music.zip -d /content/music/

Streaming output truncated to the last 5000 lines.
  inflating: /content/music/MIDI/Inawhile.mid  
  inflating: /content/music/MIDI/Incantations4OdeCynthia.mid  
  inflating: /content/music/MIDI/indaclub.mid  
  inflating: /content/music/MIDI/infantlo.mid  
  inflating: /content/music/MIDI/Infected (5).mid  
  inflating: /content/music/MIDI/Innamorare tanto.mid  
  inflating: /content/music/MIDI/Intro (9).mid  
  inflating: /content/music/MIDI/intrust.mid  
  inflating: /content/music/MIDI/in_the_good_old_summertime1-Db160.mid  
  inflating: /content/music/MIDI/in_the_summertime_1.mid  
  inflating: /content/music/MIDI/io ti aspetterò.mid  
  inflating: /content/music/MIDI/iran2.mid  
  inflating: /content/music/MIDI/isntlifestrange.mid  
  inflating: /content/music/MIDI/It Came Upon A Midnight Clear (Xmas) (Seq Harry Todd) midnight_ht.mid  
  inflating: /content/music/MIDI/IT'S.MID  
  inflating: /content/music/MIDI/itsalltoomuch L.mid  
  inflating: /content/music/MIDI/i_hear_you_kno

**Convert the midis into all one track**

In [ ]:
#@title # **Convert the dataset and make sure to change midi dir if you changed it above** *Also* you may get errors but its probably ok
import py_midicsv as pm
import os
import re


midi_dir = "/content/music/" #@param {type:"string"}
midi_files = [f for f in os.listdir(midi_dir) if f.endswith('.mid')]
csv_string = []

def get_note_event(start, note, instrument, length, velocity):
  out_instrument = "%"
  match instrument:
    case 2:
      out_instrument = "%"
    case 3:
      out_instrument = "^"
    case 4:
      out_instrument = "&"
    case 5:
      out_instrument = "*"
    case 6:
      out_instrument = "("
    case 7:
      out_instrument = "("
    case _:
      out_instrument = "%"


  dynamic = "$"
  if(velocity > 108):
    dynamic = "$"
  elif velocity > 74:
    dynamic = "#"
  elif velocity > 40:
    dynamic = "@"
  else:
    dynamic = "!"

  return [start, dynamic, length, out_instrument, note]

def parse_split_csv_item(item):
  try:
    item[2] = item[2].replace(' ', '')
    item[0] = int(item[0])
    item[1] = int(item[1])
    item[3] = int(item[3])
    item[4] = int(item[4])
    item[5] = int(item[5])
  except:
    pass;
  return item

# Load the MIDI file and parse it into CSV format
numNotesInCsv = 0
for midi_file in midi_files:
  csv_parsed = []
  try:
    lastNoteStart = 0
    csv_unparsed = pm.midi_to_csv(midi_dir + midi_file)
    notesPlaying = []
    for csv_item in csv_unparsed:
      split_csv_item = csv_item.split(',')
      split_csv_item = parse_split_csv_item(split_csv_item);
      match(split_csv_item[2]):
        case "Note_on_c":
          toAdd = True
          for i in notesPlaying:
            if(i[4] == split_csv_item[4]):
              toAdd = False
          if toAdd:
            notesPlaying.append(split_csv_item)
        case "Note_off_c":
          # Find note that turning off
          i = 0
          while i < len(notesPlaying):
            if(notesPlaying[i][4] == split_csv_item[4]):
              temp = get_note_event(notesPlaying[i][1], split_csv_item[4], split_csv_item[0], split_csv_item[1] - notesPlaying[i][1], notesPlaying[i][5])
              csv_parsed.append(temp)
              #csv_parsed.append(get_note_event(notesPlaying[i][1] - lastNoteStart, split_csv_item[4], 0, split_csv_item[1] - notesPlaying[i][1], notesPlaying[i][5]) + '|')
              #csv_string += get_note_event(notesPlaying[i][1] - lastNoteStart, split_csv_item[4], 0, split_csv_item[1] - notesPlaying[i][1], notesPlaying[i][5]) + '|'
              lastNoteStart = notesPlaying[i][1]
              notesPlaying.pop(i)
              i-=1
            i+=1
  except Exception as e:
    print(e)

  # Change to be delta time instead of abs and sort time
  csv_parsed = sorted(csv_parsed, key=lambda x: x[0])
  for i in range(len(csv_parsed) - 1):
    csv_item_copy = csv_parsed[i].copy()
    if(i != 0):
      csv_item_copy[0] = csv_parsed[i][0] - csv_parsed[i - 1][0]

    # Also concat to csv_parsed
    for j in csv_item_copy:
      csv_string += str(j)
    csv_string += "|"
    numNotesInCsv+=1
    if(numNotesInCsv % 400 == 0):
      csv_string += "\n"
  csv_string += "\n"

#for i in range(0, len(csv_string), 1800):
#  csv_string[i] = "\n"

with open("convertedData.txt", "w") as f:
    f.writelines(csv_string)

Event length mismatch for PortEvent at position 60
Event length mismatch for PortEvent at position 67
Event length mismatch for PortEvent at position 60
Event length mismatch for PortEvent at position 76
Controller value is out of range: 255 at position 1021
Event length mismatch for PortEvent at position 167


Event length mismatch for PortEvent at position 100
Event length mismatch for PortEvent at position 178
Event length mismatch for PortEvent at position 182
Event length mismatch for PortEvent at position 130
Event length mismatch for PortEvent at position 100
Event length mismatch for PortEvent at position 165
Event length mismatch for PortEvent at position 60
Event length mismatch for PortEvent at position 104
Event length mismatch for PortEvent at position 60
Event length mismatch for PortEvent at position 73
Event length mismatch for PortEvent at position 67
Event length mismatch for PortEvent at position 69
Event length mismatch for PortEvent at position 103
Event length mismatch for PortEvent at position 247
('Bad track header in MIDI file: ', b'')
Event length mismatch for PortEvent at position 126
Event length mismatch for PortEvent at position 147
Event length mismatch for PortEvent at position 1897
Event length mismatch for PortEvent at position 69
Event length mismatch for Po

Event length mismatch for PortEvent at position 181
Event length mismatch for PortEvent at position 67
Event length mismatch for PortEvent at position 39
Event length mismatch for PortEvent at position 213
Event length mismatch for PortEvent at position 61
Event length mismatch for PortEvent at position 275
Event length mismatch for PortEvent at position 81
Event length mismatch for PortEvent at position 54
Event length mismatch for PortEvent at position 60
Event length mismatch for PortEvent at position 92
Event length mismatch for PortEvent at position 60
Event length mismatch for PortEvent at position 68
Event length mismatch for PortEvent at position 164
Event length mismatch for PortEvent at position 51
Event length mismatch for PortEvent at position 82
Event length mismatch for PortEvent at position 42
Event length mismatch for PortEvent at position 27
Event length mismatch for PortEvent at position 39
Event length mismatch for PortEvent at position 219
Event length mismatch for 

Unknown Meta MIDI Event 10 at position 66


('Bad track header in MIDI file: ', b'')
Event length mismatch for PortEvent at position 192
Event length mismatch for PortEvent at position 145
Event length mismatch for PortEvent at position 60
Event length mismatch for PortEvent at position 262
Event length mismatch for PortEvent at position 366
Event length mismatch for PortEvent at position 130
Event length mismatch for PortEvent at position 115
Event length mismatch for PortEvent at position 171
Event length mismatch for PortEvent at position 491
Event length mismatch for PortEvent at position 418


...

Controller value is out of range: 128 at position 8471
Event length mismatch for PortEvent at position 104
Event length mismatch for PortEvent at position 85
Event length mismatch for PortEvent at position 72
Event length mismatch for PortEvent at position 106
Event length mismatch for PortEvent at position 74
Event length mismatch for PortEvent at position 250
Event length mismatch for PortEvent at position 83
Event length mismatch for PortEvent at position 341
Event length mismatch for PortEvent at position 54
Event length mismatch for PortEvent at position 73
Event length mismatch for PortEvent at position 195
Event length mismatch for PortEvent at position 98
Event length mismatch for PortEvent at position 219
Event length mismatch for PortEvent at position 181
Event length mismatch for PortEvent at position 38
Event length mismatch for PortEvent at position 173
Event length mismatch for PortEvent at position 330
Event length mismatch for PortEvent at position 34
Event length misma

Event length mismatch for PortEvent at position 773
Event length mismatch for PortEvent at position 91
Event length mismatch for PortEvent at position 238
Event length mismatch for PortEvent at position 242
Event length mismatch for PortEvent at position 237
Event length mismatch for PortEvent at position 60
Event length mismatch for PortEvent at position 189
Event length mismatch for PortEvent at position 122
Event length mismatch for PortEvent at position 309
Event length mismatch for PortEvent at position 318
Event length mismatch for PortEvent at position 80
Event length mismatch for PortEvent at position 194
Event length mismatch for PortEvent at position 95
Event length mismatch for PortEvent at position 107
Controller value is out of range: 203 at position 257
Event length mismatch for PortEvent at position 249
Event length mismatch for PortEvent at position 230
Event length mismatch for PortEvent at position 100


Event length mismatch for PortEvent at position 219
Event length mismatch for PortEvent at position 76
Event length mismatch for PortEvent at position 60
Event length mismatch for PortEvent at position 76
Event length mismatch for PortEvent at position 115
Event length mismatch for PortEvent at position 186
Event length mismatch for PortEvent at position 124
Event length mismatch for PortEvent at position 380
Event length mismatch for PortEvent at position 111
Event length mismatch for PortEvent at position 60
Event length mismatch for PortEvent at position 264
Event length mismatch for PortEvent at position 418
Event length mismatch for PortEvent at position 128
Event length mismatch for PortEvent at position 175
Event length mismatch for PortEvent at position 35295


Velocity value is out of range: 254 at position 289
Event length mismatch for PortEvent at position 60
Event length mismatch for PortEvent at position 211
Event length mismatch for PortEvent at position 233
Event length mismatch for PortEvent at position 83
Event length mismatch for PortEvent at position 186
Event length mismatch for PortEvent at position 64
Event length mismatch for PortEvent at position 60
Event length mismatch for PortEvent at position 706
Event length mismatch for PortEvent at position 27
Event length mismatch for PortEvent at position 219
Event length mismatch for PortEvent at position 392
Event length mismatch for PortEvent at position 124
Event length mismatch for PortEvent at position 60
Event length mismatch for PortEvent at position 226
Event length mismatch for PortEvent at position 127
Event length mismatch for PortEvent at position 1847
Event length mismatch for PortEvent at position 236
Event length mismatch for PortEvent at position 1128
Event length mis

Event length mismatch for PortEvent at position 60
Event length mismatch for PortEvent at position 88
Event length mismatch for PortEvent at position 68
Event length mismatch for PortEvent at position 267
Event length mismatch for PortEvent at position 69
